In [ ]:
import pandas as pd
import re
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

In [ ]:
url = "https://www.linkedin.com"

PROJECT_ROOT = os.path.abspath(os.getcwd())
DRIVER_BIN = os.path.join(PROJECT_ROOT, "chromedriver")

browser = webdriver.Chrome(executable_path = DRIVER_BIN)
browser.get(url)

#maximize window size
browser.maximize_window()

In [ ]:
# sleep for 0.5 seconds
time.sleep(10)
# use WebDriverWait to wait explicitly within 100 seconds limit
username = WebDriverWait(browser, 10).until(
    EC.presence_of_element_located((By.ID, "session_key"))
)

username.send_keys("") # use your own LinkedIn username to login
password = WebDriverWait(browser, 10).until(
    EC.presence_of_element_located((By.ID, "session_password"))
)
password.send_keys("") # use your own LinkedIn passcode to login

login_button = browser.find_element_by_class_name("sign-in-form__submit-button")
login_button.click()
browser.implicitly_wait(5)
browser.maximize_window()

In [ ]:
search_words = ["information science", "data management", "data science", "data analytics", "machine learning", "artificial intelligence", "information management", "information risk", "data governance", "data quality"]

search_word = search_words[0] # need to manually select the term you want to search everytime it scrapes
search_word_suffix = search_word.replace(" ", "%20")
count = 0
maxpage = 0
pages = []
while maxpage < 41:
    page = "https://www.linkedin.com/jobs/search/?keywords=" + search_word_suffix + "&start=" + str(count)
    maxpage += 1
    count += 25
    browser.get(page)
    time.sleep(30)
# pages store all the links that the web scraping needs to onboard

In [ ]:
# automate scrolling down left-page
# scroll down page until flag elements are visible
try:
    flag1 = browser.find_element_by_xpath("//ul[@class='jobs-search-results__list list-style-none']/li[9]")
    browser.execute_script("arguments[0].scrollIntoView();", flag1)
except:
    print("error1")
time.sleep(10)

try:
    flag2 = browser.find_element_by_xpath("//ul[@class='jobs-search-results__list list-style-none']/li[15]")
    browser.execute_script("arguments[0].scrollIntoView();", flag2)
except:
    print("error")
time.sleep(10)

try:
    flag3 = browser.find_element_by_xpath("//ul[@class='jobs-search-results__list list-style-none']/li[21]")
    browser.execute_script("arguments[0].scrollIntoView();", flag3)
except:
    print("error")

time.sleep(20)

In [ ]:
#repeat this
#25 records per page
job_titles = WebDriverWait(browser, 20).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "job-card-list__title"))
)
job_comps = WebDriverWait(browser, 20).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "job-card-container__company-name"))
)
job_boxs = WebDriverWait(browser, 20).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "jobs-search-results__list-item"))
)

jobs_num = len(job_titles)
print(jobs_num)

25


In [ ]:
#repeat this
jobs = []
#get data from catalogue result page - name/title/url
for i in range(jobs_num):
    title = job_titles[i]
    comp = job_comps[i]
    title_name = title.text
    title_link = title.get_attribute('href')
    comp_name = comp.text
    comp_link = comp.get_attribute('href')
    jobs.append({'keyword':search_word, 'JobTitle':title_name, 'JobUrl':title_link, 'Company':comp_name, 'CompanyUrl':comp_link})

In [ ]:
#repeat this
#use item box
for i in range(jobs_num):
    job_boxs[i].click()
    time.sleep(5)
    try:
        job = WebDriverWait(browser, 5).until(
            EC.presence_of_element_located((By.ID, "job-details"))
            )
        if job:
            jd_plain = job.text #job.text doesn't include text in <strong> and <ul>
            #jobs[i].update({'jd_plain':jd_plain})
    except:
        continue

    add_text = ''
    # if the job description contains bullet points text
    try:
        add = WebDriverWait(browser, 5).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@id='job-details']/span/ul/li"))
            )
        if add:
            for txt in add:
                add_text = add_text + ' ' + txt.get_attribute("innerHTML") #issue: txt.text doesn't work here
    except:
        continue
    finally:
        jd_all = jd_plain + ' ' + add_text
        jobs[i].update({'jdPlain':jd_plain, 'jdBullet':add_text, 'jdFull':jd_all})



In [ ]:
"""test only"""
for i in range(len(jobs)):
    print("heyyyyyyyyy", i)
    print(jobs[i]['jdBullet'], " ")

heyyyyyyyyy 0
  As a member of the team, you will gain satisfaction though adding value and contributing to the team’s initiatives  Help define new solutions that will improve data governance  Help define and source metrics that help us measure data management maturity from varying perspectives – committee, data office, data governance.  Partner with Divisional Data Offices to help them adopt the governance framework and utilize governancies tools:  Inform/train them how to implement the policies and monitor data quality/classification within their areas  Understand their tooling needs and help specify the next generation of solutions  Help with team planning, program reporting and communication  Help drive the evolution and maturity of governance  Grow your business understanding of data  Develop business analysis and relationship management skills  Contribute to setting a course for the future of data management and usage at Goldman Sachs  Ability to effectively communicate and prese

In [ ]:
# repeat this
# write dict data into csv file
columns = ['keyword', 'JobTitle', 'JobUrl', 'Company', 'CompanyUrl', 'jdPlain', 'jdBullet', 'jdFull']
csv_file = "LinkedIn_dataset_full_scroll2.csv"

with open(csv_file, 'a+', encoding='utf-8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=columns)
    reader = csv.reader(f)
    if not [row for row in reader]:
        writer.writeheader()
        for job_record in jobs:
            writer.writerow(job_record)
    else:
        for job_record in jobs:
            writer.writerow(job_record)

